<a href="https://colab.research.google.com/github/chabryl/AI-Redirect-Mapping/blob/main/redirect_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Benötigte Pakete installieren
!pip install sentence-transformers qdrant-client pandas

import pandas as pd
import numpy as np
import os
import time
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from google.colab import files, userdata

In [4]:
# Datei zum Speichern der Laufzeiten
report_file = 'runtime_report.txt'

# Startzeit für die Gesamtzeitmessung
total_start_time = time.time()

with open(report_file, 'w') as f:
    f.write("### Laufzeitbericht ###\n\n")

    # Schritt 1: Zeitmessung für CSV-Dateien hochladen
    f.write("Modul 1: Hochladen der CSV-Dateien\n")
    start_time = time.time()

    uploaded = files.upload()
    crawl_file = 'crawl_all.csv'
    error_file = '404_errors.csv'

    end_time = time.time()
    f.write(f"Zeit für Modul 1: {end_time - start_time:.2f} Sekunden\n\n")

    # Schritt 2: Zeitmessung für CSV-Dateien laden
    f.write("Modul 2: Laden der CSV-Dateien\n")
    start_time = time.time()

    crawl_data = pd.read_csv(crawl_file)
    error_data = pd.read_csv(error_file)

    end_time = time.time()
    f.write(f"Zeit für Modul 2: {end_time - start_time:.2f} Sekunden\n\n")

    # Schritt 3: Zeitmessung für das Erstellen und Speichern der Embeddings
    f.write("Modul 3: Erstellen und Speichern der Embeddings\n")
    start_time = time.time()

    def create_and_save_embeddings(df, save_path, model_name="paraphrase-multilingual-MiniLM-L12-v2"):
        if not os.path.exists(save_path):
            # Lade das vortrainierte Modell
            model = SentenceTransformer(model_name)

            # Erstelle eine Liste der Texte (Title, Meta Description, H1)
            texts = df.apply(lambda row: f"{row['Title']} {row['Meta Description']} {row['H1']}", axis=1).tolist()

            # Generiere Embeddings
            embeddings = model.encode(texts)

            # Speichere die Embeddings und die URLs in eine Datei
            np.savez(save_path, embeddings=embeddings, urls=df['url'].tolist())
            print(f"Embeddings erfolgreich in {save_path} gespeichert.")
        else:
            print(f"Embeddings bereits in {save_path} vorhanden.")

    embeddings_file = 'crawl_embeddings.npz'
    create_and_save_embeddings(crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2")

    end_time = time.time()
    f.write(f"Zeit für Modul 3: {end_time - start_time:.2f} Sekunden\n\n")

    # Schritt 4: Zeitmessung für die Verbindung zu Qdrant
    f.write("Modul 4: Verbindung zur Qdrant-Datenbank\n")
    start_time = time.time()

    # Qdrant API-Details (aus Colab Userdata)
    api_key_qudrant = userdata.get('api_key_qudrant')
    api_url_qudrant = userdata.get('api_url_qudrant')

    qdrant_client = QdrantClient(url=api_url_qudrant, api_key=api_key_qudrant)

    end_time = time.time()
    f.write(f"Zeit für Modul 4: {end_time - start_time:.2f} Sekunden\n\n")

    # Schritt 5: Zeitmessung für das Hochladen der Embeddings zu Qdrant
    f.write("Modul 5: Hochladen der Embeddings zu Qdrant\n")
    start_time = time.time()

    def upload_embeddings_to_qdrant(client, collection_name, embeddings_file):
        # Lade die Embeddings und die zugehörigen URLs
        data = np.load(embeddings_file)
        embeddings = data['embeddings']
        urls = data['urls']

        # Überprüfe, ob die Sammlung bereits existiert, und erstelle sie falls nicht
        try:
            client.get_collection(collection_name)
            print(f"Die Sammlung {collection_name} existiert bereits.")
        except:
            print(f"Die Sammlung {collection_name} wird neu erstellt.")
            client.recreate_collection(
                collection_name=collection_name,
                vectors_config=models.VectorParams(
                    size=embeddings.shape[1],  # Größe des Embeddings
                    distance=models.Distance.COSINE
                )
            )
            client.upload_collection(
                collection_name=collection_name,
                vectors=embeddings,
                payload=[{"url": url} for url in urls]
            )
            print(f"Embeddings erfolgreich in der Sammlung {collection_name} hochgeladen.")

    collection_name = "crawl_data_collection"
    upload_embeddings_to_qdrant(qdrant_client, collection_name, embeddings_file)

    end_time = time.time()
    f.write(f"Zeit für Modul 5: {end_time - start_time:.2f} Sekunden\n\n")

    # Schritt 6: Zeitmessung für die Ähnlichkeitssuche und das Speichern der Ergebnisse
    f.write("Modul 6: Ähnlichkeitssuche und Speichern der Ergebnisse\n")
    start_time = time.time()

    def find_similar_urls_for_404(client, collection_name, error_data, crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2"):
        # Lade das Modell für die Embedding-Erstellung
        model = SentenceTransformer(model_name)

        # Lade die Embeddings aus der Datei
        data = np.load(embeddings_file)
        embeddings = data['embeddings']
        urls = data['urls']

        # Liste zur Speicherung der Ergebnisse
        results_list = []

        for index, row in error_data.iterrows():
            url_404 = row['url']

            # Suche die Daten zur 404-URL in den Crawldaten
            matching_row = crawl_data[crawl_data['url'] == url_404]

            if matching_row.empty:
                print(f"Keine Daten für die 404-URL {url_404} gefunden")
                continue

            # Erstelle Embedding für die 404-Fehlerseite (basierend auf Title, Meta Description, H1)
            text_404 = f"{matching_row['Title'].values[0]} {matching_row['Meta Description'].values[0]} {matching_row['H1'].values[0]}"
            embedding_404 = model.encode([text_404])[0]

            # Suche ähnliche URLs in der Qdrant Datenbank (aber nicht identische URLs)
            results = client.search(
                collection_name=collection_name,
                query_vector=embedding_404,
                limit=5
            )

            # Sammle die Ergebnisse für die CSV-Ausgabe
            similar_urls_with_scores = [(hit.payload['url'], hit.score) for hit in results if hit.payload['url'] != url_404]

            if similar_urls_with_scores:
                # Höchste Übereinstimmung finden
                highest_match_url, highest_score = max(similar_urls_with_scores, key=lambda x: x[1])

                # Kombiniere die ähnlichen URLs mit ihren Scores in eine Liste
                similar_urls_str = ', '.join([f"{url} (Score: {score:.4f})" for url, score in similar_urls_with_scores])

                # Speichere die 404 URL, die ähnlichen URLs und die höchste Übereinstimmung in der Liste
                results_list.append({
                    '404_url': url_404,
                    'similar_urls_with_scores': similar_urls_str,
                    'highest_match': f"{highest_match_url} (Score: {highest_score:.4f})"
                })

        # Konvertiere die Ergebnisse in einen DataFrame und speichere sie als CSV
        results_df = pd.DataFrame(results_list)
        results_df.to_csv('similar_urls_for_404.csv', index=False)
        print("Ergebnisse erfolgreich in 'similar_urls_for_404.csv' gespeichert.")

    find_similar_urls_for_404(qdrant_client, collection_name, error_data, crawl_data, embeddings_file, model_name="paraphrase-multilingual-MiniLM-L12-v2")

    end_time = time.time()
    f.write(f"Zeit für Modul 6: {end_time - start_time:.2f} Sekunden\n\n")

    # Gesamtzeit messen
    total_end_time = time.time()
    f.write(f"Gesamtlaufzeit des Skripts: {total_end_time - total_start_time:.2f} Sekunden\n")

print(f"Laufzeitbericht wurde in '{report_file}' gespeichert.")
files.download(report_file)
files.download('similar_urls_for_404.csv')


Saving 404_errors.csv to 404_errors.csv
Saving crawl_all.csv to crawl_all.csv


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

...

Embeddings erfolgreich in crawl_embeddings.npz gespeichert.
Die Sammlung crawl_data_collection existiert bereits.
Keine Daten für die 404-URL /de/flachdach/absturzsicherung/lichtkuppelsicherung.htmlLichtkuppeldurchsturzsicherung nach DGUV Information 203-058LIGHT-FLEXDie größenvariable kollektive Absturzsicherung LIGHT-FLEX ist die robuste Lösung für verschiedenste Lichtkuppeltypen sowie Lichtbänder. Das System überzeugt durch eine rasche und unkomplizierte Montage und ist auf Holz, Beton, Stahl und Kunststoff möglich.flexible Durchsturzsicherung bei offener und geschlossener Lichtkuppeleinfacher Einbau, einfaches Nachrüstenauch für Rauch-Wärmeabzugsanlagen (RWA)Teilausschnitt für Öffnungsmechanismusvariabel ausführbarZertifizierung nach dem neuesten Stand der Technik:GS-BAU-18, EN 1873:2016, EN 14963:2006 gefunden
Ergebnisse erfolgreich in 'similar_urls_for_404.csv' gespeichert.
Laufzeitbericht wurde in 'runtime_report.txt' gespeichert.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>